In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchtext
from torchtext.legacy import datasets

import numpy as np
import time
import os
import string
import random

In [25]:
TEXT = torchtext.legacy.data.Field(lower=True,  fix_length=200, batch_first=True)
LABEL = torchtext.legacy.data.Field(sequential=False)

- train/val/test data 분리

In [26]:
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [27]:
print(vars(train_data.examples[0]))

{'text': ['bromwell', 'high', 'is', 'a', 'cartoon', 'comedy.', 'it', 'ran', 'at', 'the', 'same', 'time', 'as', 'some', 'other', 'programs', 'about', 'school', 'life,', 'such', 'as', '"teachers".', 'my', '35', 'years', 'in', 'the', 'teaching', 'profession', 'lead', 'me', 'to', 'believe', 'that', 'bromwell', "high's", 'satire', 'is', 'much', 'closer', 'to', 'reality', 'than', 'is', '"teachers".', 'the', 'scramble', 'to', 'survive', 'financially,', 'the', 'insightful', 'students', 'who', 'can', 'see', 'right', 'through', 'their', 'pathetic', "teachers'", 'pomp,', 'the', 'pettiness', 'of', 'the', 'whole', 'situation,', 'all', 'remind', 'me', 'of', 'the', 'schools', 'i', 'knew', 'and', 'their', 'students.', 'when', 'i', 'saw', 'the', 'episode', 'in', 'which', 'a', 'student', 'repeatedly', 'tried', 'to', 'burn', 'down', 'the', 'school,', 'i', 'immediately', 'recalled', '.........', 'at', '..........', 'high.', 'a', 'classic', 'line:', 'inspector:', "i'm", 'here', 'to', 'sack', 'one', 'of', '

In [28]:
for exam in train_data.examples:
    text = [x.lower() for x in vars(exam)['text']]
    text = [x.replace('<br', '') for x in text]
    text = [''.join(c for c in s if c not in string.punctuation) for s in text]
    text = [s for s in text if s]
    vars(exam)['text'] = text

In [29]:
train_data, val_data = train_data.split(random_state=random.seed(0), split_ratio=0.8)

In [30]:
print(f'Number of train examples: {len(train_data)}')
print(f'Number of val examples: {len(val_data)}')
print(f'Number of test examples: {len(test_data)}')

Number of train examples: 20000
Number of val examples: 5000
Number of test examples: 25000


- 단어 집합 만들기 : build_vocab
    - max_size : 단어 집합의 크기로 단어 집합에 포함되는 어휘 수
    - min_freq : 특정 단어의 최소 등장 횟수
    - vector : 임베딩 지정

In [31]:
TEXT.build_vocab(train_data, max_size=10000, min_freq=10, vectors=None)
LABEL.build_vocab(train_data)

print(f'Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}')
print(f'Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}')

Unique tokens in TEXT vocabulary: 10002
Unique tokens in LABEL vocabulary: 3


In [32]:
print(LABEL.vocab.stoi)

defaultdict(<bound method Vocab._default_unk_index of <torchtext.legacy.vocab.Vocab object at 0x00000166153E2A90>>, {'<unk>': 0, 'pos': 1, 'neg': 2})


In [33]:
BATCH_SIZE = 100
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [34]:
train_dataloader, val_dataloader, test_dataloader = torchtext.legacy.data.BucketIterator.splits((train_data, val_data, test_data), batch_size=BATCH_SIZE, device=DEVICE)

In [35]:
vocab_size = len(TEXT.vocab)
n_classes = 2 # 긍정/부정

In [50]:
from model import BasicRNN
from utils import train, evaluate

In [37]:
model = BasicRNN(n_layers=1, hidden_dim=256, n_vocab=vocab_size, embed_dim=128, n_classes=n_classes, dropout_p=0.5)
model.to(DEVICE)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [38]:
print(model)

BasicRNN(
  (embed): Embedding(10002, 128)
  (dropout): Dropout(p=0.5, inplace=False)
  (rnn): RNN(128, 256, batch_first=True)
  (out): Linear(in_features=256, out_features=2, bias=True)
)


In [49]:
BATCH_SIZE = 100
LR = 0.001
EPOCH = 5

for e in range(1, EPOCH+1):
    train(model=model, optimizer=optimizer, train_dataloader=train_dataloader, device=DEVICE)
    val_loss, val_accuracy = evaluate(model=model, val_dataloader=val_dataloader, device=DEVICE)

    print(f'[EPOCH: {e}], Val Loss: {val_loss} | Val Accuracy: {val_accuracy}')

Train Epoch: 1 [0/20000(0.0%)] 	Loss: 0.683027
Train Epoch: 1 [5000/20000(25.0%)] 	Loss: 0.694591
Train Epoch: 1 [10000/20000(50.0%)] 	Loss: 0.681756
Train Epoch: 1 [15000/20000(75.0%)] 	Loss: 0.700426
[EPOCH: 1], Val Loss: 0.6894723617553711 | Val Accuracy: 0.5162000060081482
Train Epoch: 2 [0/20000(0.0%)] 	Loss: 0.662277
Train Epoch: 2 [5000/20000(25.0%)] 	Loss: 0.683297
Train Epoch: 2 [10000/20000(50.0%)] 	Loss: 0.689828
Train Epoch: 2 [15000/20000(75.0%)] 	Loss: 0.666543
[EPOCH: 2], Val Loss: 0.6954123641967773 | Val Accuracy: 0.5013999938964844
Train Epoch: 3 [0/20000(0.0%)] 	Loss: 0.684097
Train Epoch: 3 [5000/20000(25.0%)] 	Loss: 0.693774
Train Epoch: 3 [10000/20000(50.0%)] 	Loss: 0.694919


KeyboardInterrupt: 

In [51]:
test_loss, test_acc = evaluate(model=model, val_dataloader=test_dataloader, device=DEVICE)
print(f'Test Loss: {test_loss:.6f} | Test Acc: {test_acc:.6f}')

Test Loss: 0.697592 | Test Acc: 0.502680
